In [1]:
import os
# import numpy as np # to process the data
import random
import math
import matplotlib.pyplot as plt
# !pip install h5py
import h5py
from random import shuffle
from PIL import Image
from sklearn.model_selection import train_test_split

In [25]:
class FlowerDataset:
    def __init__(self, directory, image_size=(80, 80)):
        self.directory = directory
        self.image_size = image_size
        self.images = []
        self.labels = []

    def load_data(self):
        image_files = os.listdir(self.directory)
        for img in image_files:
        # for img in image_files[:20]:
            try:
                img_path = os.path.join(self.directory, img)
                with Image.open(img_path) as img_array:
                    # Convert image to RGB
                    img_array = img_array.convert("RGB").resize(self.image_size)
                    img_data = [[list(img_array.getpixel((x, y))) for y in range(self.image_size[1])] for x in range(self.image_size[0])]
                    self.images.append(img_data)
                    label = img.split('_')[0]
                    self.labels.append(label)
            except Exception as e:
                print(f"Error loading image {img}: {e}")

    def preprocess_data(self):
        max_pixel_value = 255.0
        self.images = [[[[value / max_pixel_value for value in pixel] for pixel in row] for row in image] for image in self.images]

    def print_dataset_dimensions(self):
        if self.images:
            print(f"Total number of images: {len(self.images)}")
            print(f"Dimensions of each image: ({len(self.images[0])}, {len(self.images[0][0])}, {len(self.images[0][0][0])})")
            print(f"Total dimensions of the dataset (assuming all images have the same shape): {len(self.images)} x ({len(self.images[0])}, {len(self.images[0][0])}, {len(self.images[0][0][0])})")
        else:
            print("The dataset is empty or not loaded properly.")

    def one_hot_encode(self):
        unique_labels = sorted(set(self.labels))
        label_to_int = {label: index for index, label in enumerate(unique_labels)}
        self.label_indices = [label_to_int[label] for label in self.labels]
        self.labels = [[int(i == label_index) for i in range(len(unique_labels))] for label_index in self.label_indices]

def split_data(images, labels, train_ratio, validation_ratio, test_ratio):
    combined = list(zip(images, labels))
    shuffle(combined)
    shuffled_images, shuffled_labels = zip(*combined)

    train_end = int(len(shuffled_images) * train_ratio)
    validation_end = train_end + int(len(shuffled_images) * validation_ratio)

    x_train = shuffled_images[:train_end]
    y_train = shuffled_labels[:train_end]
    x_val = shuffled_images[train_end:validation_end]
    y_val = shuffled_labels[train_end:validation_end]
    x_test = shuffled_images[validation_end:]
    y_test = shuffled_labels[validation_end:]

    return list(x_train), list(x_val), list(x_test), list(y_train), list(y_val), list(y_test)

# Usage
dataset = FlowerDataset('flowers')
dataset.load_data()
dataset.preprocess_data()
dataset.one_hot_encode()

x_train, x_val, x_test, y_train, y_val, y_test = split_data(dataset.images, dataset.labels, 0.75, 0.15, 0.10)

# print("y_train after one hot encoded: ", y_train)

# Print the dimensions of the preprocessed images
dataset.print_dataset_dimensions()

# Print the sizes of the datasets
print(f"Train set size: {len(x_train)}, Validation set size: {len(x_val)}, Test set size: {len(x_test)}")

# Print the unique classes in the training set
unique_classes = sorted(set([label.index(1) for label in y_train]))
print(f"Unique classes in the training set: {unique_classes}")

num_classes = len(unique_classes)
print(f"Number of classes for prediction: {num_classes}")


Error loading image .ipynb_checkpoints: [Errno 21] Is a directory: '/storage/student14/phucpg/WORKSPACE/flowers/.ipynb_checkpoints'
Total number of images: 733
Dimensions of each image: (80, 80, 3)
Total dimensions of the dataset (assuming all images have the same shape): 733 x (80, 80, 3)
Train set size: 549, Validation set size: 109, Test set size: 75
Unique classes in the training set: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Number of classes for prediction: 10


In [26]:
class Limiting:
    @staticmethod
    def clip(values, min_value, max_value):
        # Check if values is a list or a single value
        if isinstance(values, list):
            return [max(min(value, max_value), min_value) for value in values]
        else:
            return max(min(values, max_value), min_value)

    @staticmethod
    def get_unique_filename(base_filename, extension):
        # Start with 0 suffix and increment until a unique filename is found
        counter = 0
        unique_filename = f"{base_filename}_{counter}.{extension}"
        while os.path.exists(unique_filename):
            counter += 1
            unique_filename = f"{base_filename}_{counter}.{extension}"
        return unique_filename

    @staticmethod
    def min_max_scale(data):
        min_val = min(data)
        max_val = max(data)
        range_val = max_val - min_val
        if range_val == 0:
            return [0 for _ in data]  # Return a list of zeros if all values are the same
        scaled_data = [(value - min_val) / range_val for value in data]
        return scaled_data

    @staticmethod
    def convert_to_one_hot(predicted):
        one_hot_predicted = []
        for pred in predicted:
            # Find the index with the maximum probability
            max_index = pred.index(max(pred))
            # Create a one-hot encoded vector
            one_hot_vector = [0] * len(pred)
            one_hot_vector[max_index] = 1
            one_hot_predicted.append(one_hot_vector)
        return one_hot_predicted

    @staticmethod
    def one_hot_to_class_index(one_hot_vector):
        return one_hot_vector.index(1)

class Precision:
    def calculate(self, predicted, actual):
        tp = sum(p == a == 1 for p, a in zip(predicted, actual))
        fp = sum(p == 1 and a == 0 for p, a in zip(predicted, actual))
        return tp / (tp + fp) if tp + fp > 0 else 0

class Recall:
    def calculate(self, predicted, actual):
        tp = sum(p == a == 1 for p, a in zip(predicted, actual))
        fn = sum(p == 0 and a == 1 for p, a in zip(predicted, actual))
        return tp / (tp + fn) if tp + fn > 0 else 0

class F1Score:
    def calculate(self, predicted, actual):
        precision = Precision().calculate(predicted, actual)
        recall = Recall().calculate(predicted, actual)
        return 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

In [27]:
# Base Layer class
class Layer:
    def forward(self, input_data):
        raise NotImplementedError

    def backward(self, output_error, learning_rate):
        raise NotImplementedError

# Convolutional Layer
class ConvLayer(Layer):
    def __init__(self, num_filters, filter_size, num_channels=3, padding=0, stride=1):
        self.num_filters = num_filters
        self.filter_size = filter_size
        self.filters = [[[[random.random() for _ in range(num_channels)] for _ in range(filter_size)] for _ in range(filter_size)] for _ in range(num_filters)]
        self.padding = padding
        self.stride = stride
        
    def forward(self, input_data):
        # # Assert that the input data has the expected shape
        # assert len(input_data.shape) == 4, f"Expected input shape to be 4D, got {input_data.shape}"
        # # Ensure the input data has four dimensions: batch size, height, width, and depth
        # if len(input_data.shape) < 4:
        #     input_data = np.expand_dims(input_data, axis=0)  # Add a batch dimension
        self.last_input = input_data
        batch_size, h, w, d = len(input_data), len(input_data[0]), len(input_data[0][0]), len(input_data[0][0][0])
    
        new_h = (h + 2 * self.padding - self.filter_size) // self.stride + 1
        new_w = (w + 2 * self.padding - self.filter_size) // self.stride + 1
        output = [[[[0 for _ in range(self.num_filters)] for _ in range(new_w)] for _ in range(new_h)] for _ in range(batch_size)]
    
        for b in range(batch_size):
            for i in range(0, new_h, self.stride):
                for j in range(0, new_w, self.stride):
                    for f in range(self.num_filters):
                        h_start = i*self.stride
                        h_end = h_start+self.filter_size
                        w_start = j*self.stride
                        w_end = w_start+self.filter_size
    
                        patch = [[[input_data[b][x][y][c] for c in range(d)] for y in range(w_start, w_end)] for x in range(h_start, h_end)]
                        output_value = sum(sum(sum(a*b for a, b in zip(patch_row, filter_row)) for patch_row, filter_row in zip(patch_channel, filter_channel)) for patch_channel, filter_channel in zip(patch, self.filters[f]))
                        # output[b][i][j][f] = Limiting.clip(output_value, 1e - 9, 1 - (1e -9))  # Clip the output value using Limiting.clip
                        output[b][i][j][f] = output_value
    
        return output

    def backward(self, output_error, learning_rate):
        self.last_input = self.last_input
        batch_size, h, w, d = len(self.last_input), len(self.last_input[0]), len(self.last_input[0][0]), len(self.last_input[0][0][0])
    
        new_h = (h + 2 * self.padding - self.filter_size) // self.stride + 1
        new_w = (w + 2 * self.padding - self.filter_size) // self.stride + 1
        output = [[[[0 for _ in range(self.num_filters)] for _ in range(new_w)] for _ in range(new_h)] for _ in range(batch_size)]
    
        for b in range(batch_size):
            for i in range(0, new_h, self.stride):
                for j in range(0, new_w, self.stride):
                    for f in range(self.num_filters):
                        h_start = i*self.stride
                        h_end = h_start+self.filter_size
                        w_start = j*self.stride
                        w_end = w_start+self.filter_size
    
                        patch = [[self.last_input[b][x][y] for y in range(w_start, w_end)] for x in range(h_start, h_end)]
                        output_value = sum(sum(sum(a*b for a, b in zip(patch_row, filter_row)) for patch_row, filter_row in zip(patch_channel, filter_channel)) for patch_channel, filter_channel in zip(patch, self.filters[f]))
                        output[b][i][j][f] = Limiting.clip(output_value, -10**9, 10**9)  # Clip the output value using Limiting.clip
    
        return output
        
# ReLU Activation
class ReLULayer(Layer):
    def forward(self, input_data):
        self.last_input = input_data
        return self.apply_relu(input_data)

    def backward(self, output_error, learning_rate):
        return self.apply_relu_gradient(output_error)

    def apply_relu(self, data):
        # Apply ReLU to each value in the input data
        return [[[[max(0, val) for val in channel] for channel in depth] for depth in width] for width in data]

    def apply_relu_gradient(self, error):
        # Apply the gradient of ReLU to the output error
        return [[[[err * (val > 0) for err, val in zip(channel_err, channel_val)]
                  for channel_err, channel_val in zip(depth_err, depth_val)]
                 for depth_err, depth_val in zip(width_err, width_val)]
                for width_err, width_val in zip(error, self.last_input)]

# MaxPooling Layer
class MaxPoolingLayer(Layer):
    def __init__(self, pool_size):
        self.pool_size = pool_size

    def forward(self, input_data):
        self.last_input = input_data
        batch_size, h, w, num_filters = len(input_data), len(input_data[0]), len(input_data[0][0]), len(input_data[0][0][0])
        output = [[[[0 for _ in range(num_filters)] for _ in range(w // self.pool_size)] for _ in range(h // self.pool_size)] for _ in range(batch_size)]
        for b in range(batch_size):
            for i in range(h // self.pool_size):
                for j in range(w // self.pool_size):
                    for f in range(num_filters):
                        patch = [[input_data[b][x][y][f] for y in range(j*self.pool_size, (j+1)*self.pool_size)] for x in range(i*self.pool_size, (i+1)*self.pool_size)]
                        max_value = max(max(row) for row in patch)
                        # output[b][i][j][f] = Limiting.clip(max_value, 1e - 9 , 1 - (1e - 9))  # Clip the output value using Limiting.clip
                        output[b][i][j][f] = max_value
        return output

    def backward(self, output_error, learning_rate):
        # Initialize d_input with zeros, having the same shape as the last input
        d_input = [[[[
            0 for _ in range(len(self.last_input[0][0][0]))] 
            for _ in range(len(self.last_input[0][0]))] 
            for _ in range(len(self.last_input[0]))] 
            for _ in range(len(self.last_input))]

        for b in range(len(self.last_input)):
            for h in range(0, len(self.last_input[b]), self.pool_size):
                for w in range(0, len(self.last_input[b][h]), self.pool_size):
                    for c in range(len(self.last_input[b][h][w])):
                        # Find the max value in the pooling window
                        max_value = None
                        max_h = 0
                        max_w = 0
                        for i in range(self.pool_size):
                            for j in range(self.pool_size):
                                current_h = h + i
                                current_w = w + j
                                if current_h < len(self.last_input[b]) and current_w < len(self.last_input[b][current_h]):
                                    if max_value is None or self.last_input[b][current_h][current_w][c] > max_value:
                                        max_value = self.last_input[b][current_h][current_w][c]
                                        max_h = current_h
                                        max_w = current_w
                        # Propagate the error to the max location
                        if h//self.pool_size < len(output_error[b]) and w//self.pool_size < len(output_error[b][h//self.pool_size]):
                            d_input[b][max_h][max_w][c] = output_error[b][h//self.pool_size][w//self.pool_size][c]

        return d_input
        
class FlattenLayer(Layer):
    def forward(self, input_data):
        self.last_input_shape = [len(input_data), len(input_data[0]), len(input_data[0][0]), len(input_data[0][0][0])]
        flattened = []
        
        for b in range(self.last_input_shape[0]):
            for i in range(self.last_input_shape[1]):
                for j in range(self.last_input_shape[2]):
                    for k in range(self.last_input_shape[3]):
                        value = input_data[b][i][j][k]
                        # clipped_value = Limiting.clip(value, -10**6, 10**6)  # Clip the value using Limiting.clip
                        # flattened.append(clipped_value)
                        flattened.append(value)
                        
        return [flattened]  # Return a list of lists
        
    def backward(self, output_error, learning_rate):
        reshaped_error = []
        
        for b in range(self.last_input_shape[0]):
            image_error = []
            
            for i in range(self.last_input_shape[1]):
                row_error = []
                
                for j in range(self.last_input_shape[2]):
                    pixel_error = []
                    
                    for k in range(self.last_input_shape[3]):
                        error_value = output_error[0][b*self.last_input_shape[1]*self.last_input_shape[2]*self.last_input_shape[3] + i*self.last_input_shape[2]*self.last_input_shape[3] + j*self.last_input_shape[3] + k]
                        pixel_error.append(error_value)
                        
                    row_error.append(pixel_error)
                image_error.append(row_error)
            reshaped_error.append(image_error)
            
        return reshaped_error
        
# Dropout Layer
class DropoutLayer(Layer):
    def __init__(self, dropout_rate):
        self.dropout_rate = dropout_rate

    def forward(self, input_data):
        self.mask = self.create_dropout_mask(input_data)
        return self.apply_dropout_mask(input_data)

    def backward(self, output_error, learning_rate):
        return self.apply_dropout_mask(output_error)

    def create_dropout_mask(self, input_data):
        mask = []
        for i in range(len(input_data)):
            mask_row = self.create_mask_row(len(input_data[0]))
            mask.append(mask_row)
        return mask

    def create_mask_row(self, length):
        return [(random.random() > self.dropout_rate) / (1.0 - self.dropout_rate) for _ in range(length)]

    def apply_dropout_mask(self, input_data):
        output = []
        for i in range(len(input_data)):
            output_row = self.apply_mask_row(i, input_data[i])
            output.append(output_row)
        return output

    def apply_mask_row(self, i, input_row):
        return [self.mask[i][j] * input_row[j] for j in range(len(input_row))]

# Dense Layer
class DenseLayer(Layer):
    def __init__(self, name, input_size, output_size):
        self.name = name
        self.weights = [[random.random() for _ in range(output_size)] for _ in range(input_size)]
        self.biases = [0 for _ in range(output_size)]  # Ensure biases is a 1D array with length 'output_size'
        
    def forward(self, input_data):
        # Assert that the input data is 2D after flattening
        # assert len(input_data.shape) == 2, f"Expected input shape to be 2D, got {input_data.shape}"
        self.last_input = input_data
        output = []
        for data in input_data:
            output_row = []
            for w, b in zip(self.weights, self.biases):
                value = 0
                for d, w_ in zip(data, w):
                    value += d * w_
                value += b
                output_row.append(value)
            output.append(output_row)
        return output
    
    def backward(self, output_error, learning_rate):
        d_weights = []
        for j in range(len(self.weights)):
            d_weights_row = []
            for k in range(len(self.weights[0])):
                d_weight = 0
                for i in range(len(self.last_input)):
                    d_weight += self.last_input[i][j] * output_error[i][k]
                d_weights_row.append(d_weight)
            d_weights.append(d_weights_row)
    
        d_biases = []
        for j in range(len(self.biases)):
            d_bias = 0
            for i in range(len(output_error)):
                d_bias += output_error[i][j]
            d_biases.append(d_bias)
    
        for j in range(len(self.weights)):
            for k in range(len(self.weights[0])):
                self.weights[j][k] -= learning_rate * d_weights[j][k]
    
        for j in range(len(self.biases)):
            self.biases[j] -= learning_rate * d_biases[j]
    
        d_input = []
        for i in range(len(output_error)):
            d_input_row = []
            for j in range(len(self.weights)):
                d_input_value = 0
                for k in range(len(self.weights[0])):
                    d_input_value += self.weights[j][k] * output_error[i][k]
                d_input_row.append(d_input_value)
            d_input.append(d_input_row)
    
        return d_input
        
class SoftmaxLayer(Layer):
    def forward(self, input_data):
        # # Reshape input_data to 2D if it's 1D
        # if input_data.ndim == 1:
        #     input_data = input_data.reshape(1, -1)

        self.last_input = input_data
        output = []
        for data in input_data:
            max_value = max(data)
            exp_values = [math.exp(value - max_value) for value in data]  # Subtract max_value for numerical stability
            sum_exp_values = sum(exp_values)
            softmax_values = [exp_value / sum_exp_values for exp_value in exp_values]
            output.append(softmax_values)
        return output

    def backward(self, output_error, learning_rate):
        return output_error  # Error is passed straight through to the next layer

# Cross-Entropy Loss
class CrossEntropyLoss:
    def calculate_loss(self, predicted, actual):
        # print(predicted)
        # print(actual)
        total_loss = 0
        for pred, act in zip(predicted, actual):
            clipped_pred = [Limiting.clip(p, 1e-9, 1 - 1e-9) for p in pred]  # Clip the predicted values
            loss = -sum(act_i * math.log(pred_i) for act_i, pred_i in zip(act, clipped_pred))
            # loss = -sum(act_i * math.log(pred_i) for act_i, pred_i in zip(act, pred))
            total_loss += loss
        return total_loss
        
    def calculate_gradient(self, predicted, actual):
        # print(predicted)
        # print(actual)
        gradients = []
        for pred, act in zip(predicted, actual):
            gradient = [p - a for p, a in zip(pred, act)]
            gradients.append(gradient)
        return gradients

In [28]:
class CNN:
    def __init__(self, num_classes, epochs=10, learning_rate=0.001, threshold=0.5, early_stopping_patience=5):
        self.layers = [
            ConvLayer(num_filters=32, filter_size=3, num_channels=3),  # num_channels is 3 for RGB images
            ReLULayer(),
            MaxPoolingLayer(pool_size=2),
            # ConvLayer(num_filters=64, filter_size=3, num_channels=32),  # num_channels is num_filters of previous ConvLayer
            # ReLULayer(),
            # MaxPoolingLayer(pool_size=2),
            # ConvLayer(num_filters=128, filter_size=3, num_channels=64),  # num_channels is num_filters of previous ConvLayer
            # ReLULayer(),
            # MaxPoolingLayer(pool_size=2),
            # ConvLayer(num_filters=256, filter_size=3, num_channels=128),  # num_channels is num_filters of previous ConvLayer
            # ReLULayer(),
            # MaxPoolingLayer(pool_size=2),
            # ConvLayer(num_filters=512, filter_size=3, num_channels=256),  # num_channels is num_filters of previous ConvLayer
            # ReLULayer(),
            # MaxPoolingLayer(pool_size=2),
            FlattenLayer(),
            # None, # -6   # Placeholder for first DenseLayer, will initialize properly later
            # ReLULayer(),
            # DropoutLayer(dropout_rate=0.1),
            # None, # -3   # Placeholder for second DenseLayer, will initialize properly later
            # ReLULayer(),
            DropoutLayer(dropout_rate=0.1),
            None, # -1   # Placeholder for third DenseLayer, will initialize properly later
            SoftmaxLayer()
        ] 
        self.loss = CrossEntropyLoss()
        self.num_classes = num_classes
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.threshold = threshold
        self.early_stopping_patience = early_stopping_patience
        self.best_loss = float('inf')
        self.patience_counter = 0

    def forward(self, X):
        for layer in self.layers:
            if layer is not None:
                # Ensure X is a NumPy array
                # if isinstance(X, list):
                #     X = np.array(X)
                # print(f"+ Before forward - {type(layer).__name__}: {self.get_shape(X)}")
                X = layer.forward(X)
                # print(f"- After forward - {type(layer).__name__}: {self.get_shape(X)}")
        return X

    def backward(self, output_error, learning_rate):
        for layer in reversed(self.layers):
            if layer is not None:
                # print(f"+ Before backward - {type(layer).__name__}: {self.get_shape(output_error)}")
                output_error = layer.backward(output_error, learning_rate)
                # print(f"- After backward - {type(layer).__name__}: {self.get_shape(output_error)}")
        return output_error

    def get_shape(self, lst):
        shape = []
        while isinstance(lst, list):
            shape.append(len(lst))
            lst = lst[0] if lst else None
        return tuple(shape)
        
    def reset(self):
        for layer in self.layers:
            if hasattr(layer, 'last_input'):
                layer.last_input = None

    def validate(self, X_val, y_val):
        val_losses = []
        correct_predictions = 0
        total_predictions = 0
    
        # Iterate over each example in the validation set
        for i in range(len(X_val)):
            # Forward pass
            input_data = [X_val[i]]  # Add a batch dimension
            output = self.forward(input_data)
    
            # Convert one-hot encoded vector to class index for the actual class
            actual_class_index = y_val[i].index(1)
            # Convert the output probabilities to class index for the predicted class
            predicted_class_index = max(range(len(output[0])), key=output[0].__getitem__)
    
            # Calculate loss for the current validation example
            val_loss = self.loss.calculate_loss([output[0]], [y_val[i]])  # Pass the entire one-hot encoded list
            val_losses.append(val_loss)
    
            # Check if the prediction is correct
            predicted_class = output[0].index(max(output[0]))
            # print("predicted_class: ", predicted_class)
            actual_class = y_val[i].index(1)
            # print("actual_class: ", actual_class)
            if predicted_class == actual_class:
                correct_predictions += 1
            total_predictions += 1

            # print("correct_predictions", correct_predictions)
            # print("total_predictions", total_predictions)
        
        avg_val_loss = sum(val_losses) / len(val_losses)
        avg_val_accuracy = correct_predictions / total_predictions
    
        return avg_val_loss, avg_val_accuracy
    
    def calculate_loss(self, X, y):
        return self.loss.calculate_loss(X, y)

    def calculate_accuracy(self, X, y):
        correct_predictions = 0
        for pred, act in zip(X, y):  # Use X and y instead of predicted and actual
            predicted_class = pred.index(max(pred))
            actual_class = act.index(1)
            if predicted_class == actual_class:
                correct_predictions += 1
        accuracy = correct_predictions / len(y)
        return accuracy

    def train(self, X_train, y_train, X_val, y_val):
        print(f"Initial training data shape: {self.get_shape(X_train[0])}\n")
        
        # Calculate the output size after the convolution and pooling layers
        sample_output = self.forward([X_train[0]])  # Add a batch dimension
        flattened_size = len(sample_output[0])
        
        # Initialize the DenseLayers with the correct input size
        # self.layers[-6] = DenseLayer(input_size=flattened_size, output_size=128)  # First DenseLayer
        # self.layers[-3] = DenseLayer(input_size=128, output_size=64)  # Second DenseLayer
        # self.layers[-1] = DenseLayer(input_size=64, output_size=self.num_classes)  # Third DenseLayer
        
        self.layers[-2] = DenseLayer(name='dense_layer_1', input_size=flattened_size, output_size=self.num_classes)  # Last DenseLayer

        print("\n  |====== Start training ======| \n")

        train_losses = []
        train_accuracies = []
        val_losses = []
        val_accuracies = []
        
        for epoch in range(self.epochs):
            batch_losses = []
            batch_accuracies = []
            correct_predictions = 0
            total_predictions = 0
            
            for j in range(len(X_train)):
                # print(f"\n=== start round {j}/{len(X_train)-1}/{epoch} ===\n")
                
                # forward
                input_data = [X_train[j]]  # Add a batch dimension
                output = self.forward(input_data)

                # print("---------------------------------------")
                
                # backward
                output_error = self.loss.calculate_gradient([output[0]], [y_train[j]])  # Pass the entire one-hot encoded list
                output_error = self.backward(output_error, self.learning_rate)
                                
                # Calculate loss after the backward pass
                batch_loss = self.loss.calculate_loss([output[0]], [y_train[j]])  # Pass the entire one-hot encoded list
                batch_losses.append(batch_loss)
                # print(f"{j} batch_loss {batch_loss}")

                # Calculate accuracy after the backward pass
                batch_accuracy = self.calculate_accuracy([output[0]], [y_train[j]])
                batch_accuracies.append(batch_accuracy)
                # print(f"{j} batch_accuracy {batch_accuracy}")
                # correct_predictions += batch_accuracy
                # total_predictions += 1

                # predicted_class = output[0].index(max(output[0]))
                # actual_class = y_train[j].index(1)
                # if predicted_class == actual_class:
                #     correct_predictions += 1
                # total_predictions += 1

                # print(f"{j} correct_predictions: {correct_predictions}")
                # print(f"{j} total_predictions {total_predictions}")
                
                self.reset()  # Reset the network's state after each forward pass
                
                # print(f"\n=== end round {j}/{len(X_train)-1}/{epoch} ===\n")

            # print("correct_predictions: ", batch_accuracies)
            # print("total_predictions", len(batch_accuracies))
            
            # Calculate the average training loss and accuracy for the epoch
            avg_train_loss = sum(batch_losses) / len(batch_losses)
            avg_train_accuracy = sum(batch_accuracies) / len(batch_accuracies)

            # Append the average metrics to the lists that track them across epochs
            train_losses.append(avg_train_loss)
            train_accuracies.append(avg_train_accuracy)
            
            self.reset()

            # Validation step
            val_loss, val_accuracy = self.validate(X_val, y_val)
            val_losses.append(val_loss)
            val_accuracies.append(val_accuracy)
            
            print(f"Epoch: {epoch}, Train Loss: {avg_train_loss}, Train Accuracy: {avg_train_accuracy}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}")
    
            if val_loss < self.best_loss:
                self.best_loss = val_loss
                self.patience_counter = 0
            else:
                self.patience_counter += 1
            if self.patience_counter >= self.early_stopping_patience:
                print("Early stopping due to no improvement in validation loss.")
                break

        print()
        print("train_losses:", train_losses)
        print("train_accuracies:", train_accuracies)
        print("val_losses:", val_losses)
        print("val_accuracies:", val_accuracies)
        print()
        
        print("Training finished.")
        print(f"Best validation accuracy achieved: {self.best_loss}")

        self.plot_results(train_losses, train_accuracies, val_losses, val_accuracies)

    def predict(self, X):
        predictions = []
        for i in range(len(X)):
            # Ensure the input data has four dimensions: batch size, height, width, and depth
            input_data = [X[i]]  # Add a batch dimension
            output = self.forward(input_data)
            predictions.append(max(range(len(output[0])), key=output[0].__getitem__))  # Use max to get the class index
        return predictions
    
    def plot_results(self, train_losses, train_accuracies, val_losses, val_accuracies):
        actual_epochs = min(len(train_losses), len(train_accuracies), len(val_losses), len(val_accuracies))
        assert actual_epochs > 0, "No data to plot."
    
        # If 'val_losses' is a nested list, flatten it
        if isinstance(val_losses[0], list):
            val_losses = [loss for sublist in val_losses for loss in sublist]
        
        epochs = range(1, actual_epochs + 1)
    
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        plt.plot(epochs, train_losses[:actual_epochs], 'g', label='Training loss')
        plt.plot(epochs, val_losses[:actual_epochs], 'b', label='Validation loss')
        plt.title('Training and Validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
    
        plt.subplot(1, 2, 2)
        plt.plot(epochs, train_accuracies[:actual_epochs], 'g', label='Training accuracy')
        plt.plot(epochs, val_accuracies[:actual_epochs], 'b', label='Validation accuracy')
        plt.title('Training and Validation Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
    
        plt.tight_layout()
        plt.show()

    def save_model(self, file_name):
        with h5py.File(file_name, 'w') as f:
            for layer in self.layers:
                if hasattr(layer, 'weights'):
                    f.create_dataset(f'{layer.name}_weights', data=layer.weights)
                if hasattr(layer, 'biases'):
                    f.create_dataset(f'{layer.name}_biases', data=layer.biases)

    def load_model(self, file_name):
        with h5py.File(file_name, 'r') as f:
            for layer in self.layers:
                if hasattr(layer, 'weights'):
                    layer.weights = f[f'{layer.name}_weights'][:]
                if hasattr(layer, 'biases'):
                    layer.biases = f[f'{layer.name}_biases'][:]


In [ ]:
# Adjust the values to train
num_classes = num_classes
epochs = 100
learning_rate = 0.01
threshold = 0.5
early_stopping_patience = 15

cnn = CNN(num_classes, epochs, learning_rate, threshold, early_stopping_patience)
cnn.train(x_train, y_train, x_val, y_val)

Initial training data shape: (80, 80, 3)


  |====== Start training ======| 

Epoch: 0, Train Loss: 6.454484370271792, Train Accuracy: 0.10746812386156648, Val Loss: 7.455268345700294, Val Accuracy: 0.09174311926605505
Epoch: 1, Train Loss: 8.19942247668675, Train Accuracy: 0.1092896174863388, Val Loss: 8.890938888288735, Val Accuracy: 0.13761467889908258
Epoch: 2, Train Loss: 9.536742953794713, Train Accuracy: 0.10200364298724955, Val Loss: 11.241659193629511, Val Accuracy: 0.05504587155963303
Epoch: 3, Train Loss: 10.788523235754202, Train Accuracy: 0.08014571948998178, Val Loss: 12.126639022480187, Val Accuracy: 0.10091743119266056
Epoch: 4, Train Loss: 11.16987148592099, Train Accuracy: 0.1092896174863388, Val Loss: 10.679597936916407, Val Accuracy: 0.10091743119266056
Epoch: 5, Train Loss: 11.412722330835258, Train Accuracy: 0.10200364298724955, Val Loss: 12.62048226381551, Val Accuracy: 0.11926605504587157
Epoch: 7, Train Loss: 12.843473613685344, Train Accuracy: 0.08743169398907

In [19]:
base_filename = 'my_cnn_model'
file_extension = 'h5'
unique_filename = Limiting.get_unique_filename(base_filename, file_extension)

# Save the model
cnn.save_model(unique_filename)

In [22]:
# Later or in another script
file_name = 'my_cnn_model_0.h5'
if os.path.isfile(file_name):
    cnn_loaded = CNN(num_classes=10)
    cnn_loaded.load_model(file_name)
    print(f"Model {file_name} loaded succesfully")
else:
    print(f"The file {file_name} does not exist in the current directory.")

Model my_cnn_model_0.h5 loaded succesfully


In [ ]:
def plot_predictions(test_images, predictions, true_labels):
    plt.figure(figsize=(10, 2))
    for i in range(len(test_images)):
        plt.subplot(1, len(test_images), i+1)
        plt.imshow(test_images[i], cmap='gray')
        plt.title(f"Predicted: {predictions[i]}\nTrue: {true_labels[i]}")
        plt.axis('off')
    plt.show()